# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# import libraries
import numpy as np
import pandas as pd 

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', 'sqlite:///InsertDatabaseName.db')
X = df.message
Y = df.drop(labels=['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    return word_tokenize(text)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

In [7]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = pipeline.predict(X_test)

In [9]:
result = pd.DataFrame(Y_pred, columns=Y_test.columns)
result.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
def prediction_test(test, pred):
    for column in test.columns:
        print('==='*25, 'PREDICTION RESULT: {}\n'.format(column), sep='\n\n')
        print(classification_report(list(test[column]), list(pred[column])))

In [11]:
prediction_test(Y_test, result)


PREDICTION RESULT: related

             precision    recall  f1-score   support

          0       0.55      0.24      0.33      1550
          1       0.79      0.94      0.86      4951
          2       0.19      0.08      0.11        53

avg / total       0.73      0.77      0.73      6554


PREDICTION RESULT: request

             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5415
          1       0.76      0.55      0.64      1139

avg / total       0.88      0.89      0.88      6554


PREDICTION RESULT: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.12      0.04      0.06        26

avg / total       0.99      1.00      0.99      6554


PREDICTION RESULT: aid_related

             precision    recall  f1-score   support

          0       0.74      0.88      0.80      3815
          1       0.77      0.57      0.65      2739

avg / total       0.75 

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 1.0),
    'vect__max_features': (None, 5000)
    #'tfidf__norm' : ('l1', 'l2', None)
    #'tfidf__use_idf' : (True, False),
    #'tfidf__smooth_idf' : (True, False),
    #'tfidf__sublinear_tf' : (True, False),
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
model = cv.fit(X_train, Y_train)

In [20]:
Y_pred = model.best_estimator_.predict(X_test)

In [74]:
Y_pred_df = pd.DataFrame(Y_pred)
Y_test_df = pd.DataFrame(Y_test)

In [83]:
for i in range(Y_pred_df.shape[1]):
    print(classification_report(Y_test.values[i], Y_pred[i]))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        32
          1       1.00      1.00      1.00         4

avg / total       1.00      1.00      1.00        36

             precision    recall  f1-score   support

          0       0.96      0.96      0.96        28
          1       0.88      0.88      0.88         8

avg / total       0.94      0.94      0.94        36

             precision    recall  f1-score   support

          0       0.97      1.00      0.99        33
          1       1.00      0.67      0.80         3

avg / total       0.97      0.97      0.97        36

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
accuracy_score, precision_score, recall_score

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.